**Introduction**
Generating the Extra Columns wiith earth Engine API

In [3]:
import geopandas as gpd
import pandas as pd
import time
import os
import numpy as np
import geemap
import ee
import json
from tqdm import tqdm
import concurrent.futures

# from google.colab import auth , drive
# drive.mount('/content/drive') Only required if you are using google colab

from helper_functions import calculate_area, calculate_built_area, calculate_road_length, calculate_forest_loss
from helper_functions import calculate_elevation_and_slope, process_month, get_shadow_index_for_month, get_ndvi_for_month

In [4]:
df = gpd.read_parquet("../midsave/consolidated_reforestation_projects.parquet")
df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1072466 entries, 0 to 1072465
Data columns (total 17 columns):
 #   Column                        Non-Null Count    Dtype   
---  ------                        --------------    -----   
 0   site_id_created               1072466 non-null  int64   
 1   project_id_reported           1072466 non-null  object  
 2   site_id_reported              1072466 non-null  object  
 3   site_description_reported     1682 non-null     object  
 4   site_sqkm                     1072466 non-null  float64 
 5   trees_planted_reported        4239 non-null     float64 
 6   country                       4878 non-null     object  
 7   project_description_reported  1071904 non-null  object  
 8   planting_date_reported        1059586 non-null  float64 
 9   survival_rate_reported        2374 non-null     float64 
 10  host_name                     1072466 non-null  object  
 11  url                           1072466 non-null  object  
 12  specie

In [5]:
gdf = df.loc[:,['site_id_created', 'geometry']]

In [6]:
gdf.dropna(subset=['geometry'], inplace = True)

Create a subsample for testing code ! Delete when done !

In [8]:
gdf = df.loc[0:10000, ['site_id_created', 'geometry']]

In [9]:
gdf.dropna(subset=['geometry'], inplace = True)

### Authenticate with Google Earth Engine
Need to log in to EarthEngine (ee.Authenticate()), create a project and then initialize this project via ee.Initialize()

In [11]:
ee.Authenticate()

Enter verification code:  4/1AeanS0aRPnFFB1oLvpQhSZ5xsTG3vyqOxeXZR--YU1Xoz-iPWX94CRf9ZIw



Successfully saved authorization token.


In [12]:
ee.Initialize(project='test-reforestation')

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [13]:
# ee.Initialize(project='spring-idiom-398208')

### Calculating Tree Cover
- tree_cover_area_2000
- tree_cover_area_2005
- tree_cover_area_2010
- tree_cover_area_2015
- tree_cover_area_2020

Chunking

In [16]:
chunk_size = 1000
chunks = [gdf[i:i + chunk_size] for i in range(0, gdf.shape[0], chunk_size)]

GLAD Landcover (https://glad.umd.edu/dataset/GLCLUC2020)

In [18]:
landmask = ee.Image("projects/glad/OceanMask").lte(1)
landCover = ee.Image('projects/glad/GLCLU2020/v2/LCLUC_2020').updateMask(landmask)

Masking land cover image to only include class codes of interest for tree cover

In [20]:
classCodes = [25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48,
              125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148]
maskedLandCover = landCover.remap(classCodes, classCodes, 255)

Calculating area for each class list in the codes

In [22]:
output_csv_path = '../midsave/tree_cover.csv'
tree_cover = pd.DataFrame()

In [23]:
for i, chunk in enumerate(chunks):
    print(f"Processing chunk {i + 1}/{len(chunks)}")

    # Converting GeoDataFrame chunk to GeoJSON and then Earth Engine FeatureCollection
    gdf_json_chunk = chunk.__geo_interface__
    
    try:
        # Converting to Earth Engine FeatureCollection
        fc_chunk = geemap.geojson_to_ee(gdf_json_chunk)
    except Exception as e:
        print(f"Error converting chunk {i + 1} to Earth Engine FeatureCollection: {e}")
        continue

    # Mapping the area calculation function over the FeatureCollection
    area_results_chunk = fc_chunk.map(lambda feature: calculate_area(feature, classCodes, maskedLandCover))
    temp_chunk_df = pd.DataFrame([feature['properties'] for feature in area_results_chunk.getInfo()['features']])
    temp_chunk_df.rename(columns = {'cover_area_2020':'tree_cover_area_2020'}, inplace = True)
    # Appending to combined DataFrame
    tree_cover = pd.concat([tree_cover, temp_chunk_df], ignore_index=True)

    # Saving combined results to the output CSV
    tree_cover.to_csv(output_csv_path, index=False)

print("All chunks processed, combined results saved to:", output_csv_path)


Processing chunk 1/10


EEException: GeometryConstructors.MultiGeometry: Geometry coordinate projection requires non-zero maxError.

### Calculating other land cover
- permanent_water
- short_vegetation_after_tree_loss
- cropland_loss_to_tree
- cropland_gain_from_trees

In [ ]:
classCodes = [208, 240, 248, 245]
classesOfInterest = ["permanent_water", "short_vegetation_after_tree_loss", "cropland_loss_to_tree", "cropland_gain_from_trees"]
maskedLandCover = landCover.remap(classCodes, classCodes, 255)

In [ ]:
output_csv_path = '../midsave/other_land_cover.csv'
other_land_cover = pd.DataFrame()

In [ ]:
for i, chunk in enumerate(chunks):
    print(f"Processing chunk {i + 1}/{len(chunks)}")

    # Converting GeoDataFrame chunk to GeoJSON and then Earth Engine FeatureCollection
    gdf_json_chunk = chunk.__geo_interface__
    
    try:
        # Converting to Earth Engine FeatureCollection
        fc_chunk = geemap.geojson_to_ee(gdf_json_chunk)
    except Exception as e:
        print(f"Error converting chunk {i + 1} to Earth Engine FeatureCollection: {e}")
        continue

    # Mapping the area calculation function over the FeatureCollection
    area_results_chunk = fc_chunk.map(lambda feature: calculate_area(feature, classCodes, maskedLandCover))
    if area_results_chunk:
        temp_chunk_df = pd.DataFrame([feature['properties'] for feature in area_results_chunk.getInfo()['features']])
        temp_chunk_df.rename(columns = {'cover_area_2020':'other_land_cover_area_2020'}, inplace = True)
    else:
        temp_chunk_df = pd.DataFrame()

    other_land_cover = pd.concat([other_land_cover, temp_chunk_df], ignore_index=True)

    other_land_cover.to_csv(output_csv_path, index=False)

print("All chunks processed, combined results saved to:", output_csv_path)

### Calculate built-area shares

Loading the Earth Engine built area image for 2018

In [ ]:
builtImage = ee.Image("JRC/GHSL/P2023A/GHS_BUILT_C/2018").select('built_characteristics')

In [ ]:
output_csv_path = '../midsave/built_area_cover.csv'
built_area_cover = pd.DataFrame()

In [ ]:
for i, chunk in enumerate(chunks):
    print(f"Processing built area for chunk {i + 1}/{len(chunks)}")

    # Converting GeoDataFrame chunk to GeoJSON and then Earth Engine FeatureCollection
    gdf_json_chunk = chunk.__geo_interface__

    try:
        # Converting the GeoJSON chunk to Earth Engine FeatureCollection
        fc_chunk = geemap.geojson_to_ee(gdf_json_chunk)
    except Exception as e:
        print(f"Error converting chunk {i + 1} to Earth Engine FeatureCollection: {e}")
        continue

    # Mapping the built area calculation function over the FeatureCollection
    built_area_results_chunk = fc_chunk.map(lambda feature: calculate_built_area(feature, builtImage))
    if built_area_results_chunk:
        temp_chunk_df = pd.DataFrame([feature['properties'] for feature in built_area_results_chunk.getInfo()['features']])
    else:
        temp_chunk_df = pd.DataFrame()

    built_area_cover = pd.concat([built_area_cover, temp_chunk_df], ignore_index=True)

    built_area_cover.to_csv(output_csv_path, index=False)

print("All chunks processed, built area results saved to:", output_csv_path)

### Calculating road network cover
- total_road_length_km

Merging GEE road datasets

In [ ]:
roadsAfrica = ee.FeatureCollection('projects/ee-forest-monitoring/assets/gROADS-v1-africa')
roadsAmericas = ee.FeatureCollection('projects/ee-forest-monitoring/assets/gROADS-v1-americas')
roadsAsia = ee.FeatureCollection('projects/ee-forest-monitoring/assets/gROADS-v1-asia')
roadsEurope = ee.FeatureCollection('projects/ee-forest-monitoring/assets/gROADS-v1-europe')
roadsOceaniaEast = ee.FeatureCollection('projects/ee-forest-monitoring/assets/gROADS-v1-oceania-east')
roadsOceaniaWest = ee.FeatureCollection('projects/ee-forest-monitoring/assets/gROADS-v1-oceania-west')

roads = roadsAfrica.merge(roadsAmericas).merge(roadsAsia).merge(roadsEurope).merge(roadsOceaniaEast).merge(roadsOceaniaWest)

In [ ]:
output_csv_path = '../midsave/road_length.csv'
road_length = pd.DataFrame()

In [ ]:
for i, chunk in enumerate(chunks):
    print(f"Processing road length for chunk {i + 1}/{len(chunks)}")

    gdf_json_chunk = chunk.__geo_interface__

    try:
        # Converting the GeoJSON chunk to Earth Engine FeatureCollection
        fc_chunk = geemap.geojson_to_ee(gdf_json_chunk)
    except Exception as e:
        print(f"Error converting chunk {i + 1} to Earth Engine FeatureCollection: {e}")
        continue

    # Mapping the road length calculation function over the FeatureCollection
    road_length_results_chunk = fc_chunk.map(lambda feature: calculate_road_length(feature, roads))
    if road_length_results_chunk:
        temp_chunk_df = pd.DataFrame([feature['properties'] for feature in road_length_results_chunk.getInfo()['features']])
    else:
        temp_chunk_df = pd.DataFrame()

    road_length = pd.concat([road_length, temp_chunk_df], ignore_index=True)

    road_length.to_csv(output_csv_path, index=False)

print("All chunks processed, road length results saved to:", output_csv_path)

### Calculating forest loss
- loss_pre_5
- loss_post_3
- loss_post_5

Loading the Global Forest Change 2023 dataset

In [ ]:
gfc2017 = ee.Image('UMD/hansen/global_forest_change_2023_v1_11')

In [ ]:
output_csv_path = '../midsave/forest_loss.csv'
forest_loss = pd.DataFrame()

In [ ]:
for i, chunk in enumerate(chunks):
    print(f"Processing forest loss for chunk {i + 1}/{len(chunks)}")

    # Converting the current GeoDataFrame chunk to GeoJSON
    gdf_json_chunk = chunk.__geo_interface__

    try:
        # Converting GeoJSON chunk to Earth Engine FeatureCollection
        fc_chunk = geemap.geojson_to_ee(gdf_json_chunk)
    except Exception as e:
        print(f"Error converting chunk {i + 1} to Earth Engine FeatureCollection: {e}")
        continue

    # Mapping the forest loss calculation function over the FeatureCollection
    loss_results_chunk = fc_chunk.map(lambda feature: calculate_forest_loss(feature, gfc2017))
    if loss_results_chunk:
        temp_chunk_df = pd.DataFrame([feature['properties'] for feature in loss_results_chunk.getInfo()['features']])
    else:
        temp_chunk_df = pd.DataFrame()

    forest_loss = pd.concat([forest_loss, temp_chunk_df], ignore_index=True)

print("All chunks processed, forest loss results saved to:", output_csv_path)

#### Generating the loss columns

In [ ]:
temp = (forest_loss['groups']
               .explode()
               .dropna()
               .reset_index()
               .rename(columns = {'index':'site_id_created'})
               .reset_index())

In [ ]:
temp = (pd.json_normalize(temp['groups']).reset_index()
               .merge(temp[['index','site_id_created']], on = 'index', how = 'left').drop(columns = ['index']))

In [ ]:
temp['year'] = 2000 + temp['group']

In [ ]:
forest_loss = temp.pivot(index=['site_id_created'], columns='year', values='sum')
forest_loss.columns = [col for col in forest_loss.columns]
#forest_loss.columns = [f'forest_loss_{col}' for col in forest_loss.columns]
forest_loss.reset_index(inplace = True)

Add planting date information

In [ ]:
forest_loss = forest_loss.merge(df[['site_id_created','planting_date_reported']], on = 'site_id_created', how = 'left')

In [ ]:
def calculate_loss_pre_5(row):
    try:
        years = [int(row['planting_date_reported'] - i) for i in range(1, 6)]
        losses = [row[year] for year in years if year in forest_loss.columns]
        return np.nanmean(losses) if losses else np.nan
    except Exception as e:
        return np.nan

def calculate_loss_post_3(row):
    try:
        years = [int(row['planting_date_reported'] + i) for i in range(1, 4)]
        losses = [row[year] for year in years if year in forest_loss.columns]
        return np.nanmean(losses) if losses else np.nan
    except Exception as e:
        return np.nan

def calculate_loss_post_5(row):
    try:
        years = [int(row['planting_date_reported'] + i) for i in range(1, 6)]
        losses = [row[year] for year in years if year in forest_loss.columns]
        return np.nanmean(losses) if losses else np.nan
    except Exception as e:
        return np.nan


forest_loss['loss_pre_5'] = forest_loss.apply(calculate_loss_pre_5, axis=1)
forest_loss['loss_post_3'] = forest_loss.apply(calculate_loss_post_3, axis=1)
forest_loss['loss_post_5'] = forest_loss.apply(calculate_loss_post_5, axis=1)

In [ ]:
forest_loss.tail()

In [ ]:
forest_loss.drop(columns = [year for year in range(2000, 2024)] + ['planting_date_reported'], errors = 'ignore', inplace = True)
forest_loss.head(1)

In [ ]:
forest_loss.to_csv(output_csv_path, index=False)

### Calculating elevation and slope

Loading the Digital Elevations Model (DEM) dataset

In [ ]:
dataset = ee.Image('USGS/SRTMGL1_003')
elevation = dataset.select('elevation')
slope = ee.Terrain.slope(elevation)

In [ ]:
output_csv_path = '../midsave/elevation_slope.csv'
elevation_slope = pd.DataFrame()

In [ ]:
for i, chunk in enumerate(chunks):
    print(f"Processing chunk {i + 1}/{len(chunks)}")

    # Converting GeoDataFrame chunk to GeoJSON and then to Earth Engine FeatureCollection
    gdf_json_chunk = chunk.__geo_interface__

    try:
        fc_chunk = geemap.geojson_to_ee(gdf_json_chunk)
    except Exception as e:
        print(f"Error converting chunk {i + 1} to Earth Engine FeatureCollection: {e}")
        continue

    results_chunk = fc_chunk.map(lambda feature: calculate_elevation_and_slope(feature, elevation, slope))
    if results_chunk:
        temp_chunk_df = pd.DataFrame([feature['properties'] for feature in results_chunk.getInfo()['features']])
    else:
        temp_chunk_df = pd.DataFrame()

    # Appending to combined DataFrame
    elevation_slope = pd.concat([elevation_slope, temp_chunk_df], ignore_index=True)

    elevation_slope.to_csv(output_csv_path, index=False)

print("All chunks processed, combined results saved to:", output_csv_path)

### Calculate NDVI per month

Loading Sentinel-2 dataset

In [ ]:
S2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')

In [ ]:
output_csv_path = '../midsave/ndvi_top3.csv'
ndvi_monthly = pd.DataFrame()
months = range(1, 13)

In [ ]:
while months:
    try:
        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = {executor.submit(process_month, month, chunks, S2): month for month in months}
            
            for future in tqdm(concurrent.futures.as_completed(futures), total=len(months)):
                month_result = future.result()
                ndvi_monthly = pd.concat([ndvi_monthly, month_result], ignore_index=True)
    except Exception as e:
        print(f"Error: {e}")
        months = list(set(months) - set(ndvi_monthly.month.unique().tolist()))
        continue

Select three months with the highest NDVI values

In [ ]:
ndvi_top3 = (ndvi_monthly
             .groupby('site_id_created', group_keys=False)[['site_id_created', 'month', 'mean']]
             .apply(lambda x: x.nlargest(3, 'mean'))
             .rename(columns = {'mean':'ndvi_monthly_mean'})
             .reset_index(drop = True))

In [ ]:
ndvi_top3.to_csv(output_csv_path, index=False)

### Shadow Index 

Before shadow index ensure Top_Three_NDVI_Months and planting dates are added to df data (After creating the column Top_Three_Ndvi_months rerun cell 5 where chunks are processed to update and include the column in the chunk before running the Shadow index cell)

In [ ]:
gdf_si = (gdf
       .merge(df[['site_id_created', 'planting_date_reported']], on = 'site_id_created', how = 'left')
       .merge(ndvi_top3[['site_id_created', 'month', 'ndvi_monthly_mean']], on = 'site_id_created', how = 'left')
       .dropna()
       .reset_index(drop = True))
gdf_si['planting_date_reported'] = gdf_si['planting_date_reported'].astype(int)
gdf_si['month'] = gdf_si['month'].astype(int)

In [ ]:
chunk_size = 1000
chunks = [gdf_si[i:i + chunk_size] for i in range(0, gdf_si.shape[0], chunk_size)]

In [ ]:
output_csv_path = '../midsave/shadow_index.csv'
shadow_index = pd.DataFrame()

In [ ]:
for i, chunk in enumerate(chunks):
        print(f"Processing chunk {i + 1}/{len(chunks)}")

        gdf_json_chunk = chunk.__geo_interface__

        try:
            fc_chunk = geemap.geojson_to_ee(gdf_json_chunk)
        except Exception as e:
            print(f"Error converting chunk {i + 1} to Earth Engine FeatureCollection: {e}")
            continue
        shadow_index_chunk = fc_chunk.map(lambda feature: get_shadow_index_for_month(feature, S2))

        if results_chunk:
            temp_chunk_df = pd.DataFrame([feature['properties'] for feature in shadow_index_chunk.getInfo()['features']])
        else:
            temp_chunk_df = pd.DataFrame()
        
        # Appending to combined DataFrame
        shadow_index = pd.concat([shadow_index, temp_chunk_df], ignore_index=True)

shadow_index_grouped = shadow_index.groupby(['site_id_created'])['shadow_index'].mean().reset_index()
shadow_index_grouped.to_csv(output_csv_path, index=False)

print("All chunks processed, combined results saved to:", output_csv_path)

In [ ]:
shadow_index_grouped

### NDVI
- atplanting
- 1 year after planting
- 2 years after planting
- 5 years after planting

In [ ]:
output_csv_path = '../midsave/ndvi.csv'
ndvi = pd.DataFrame()

In [ ]:
ndvi_grouped

In [ ]:
for i, chunk in enumerate(chunks):
        print(f"Processing chunk {i + 1}/{len(chunks)}")

        gdf_json_chunk = chunk.__geo_interface__

        try:
            fc_chunk = geemap.geojson_to_ee(gdf_json_chunk)
        except Exception as e:
            print(f"Error converting chunk {i + 1} to Earth Engine FeatureCollection: {e}")
            continue
        ndvi_chunk = fc_chunk.map(lambda feature: get_ndvi_for_month(feature, S2))

        if results_chunk:
            temp_chunk_df = pd.DataFrame([feature['properties'] for feature in ndvi_chunk.getInfo()['features']])
        else:
            temp_chunk_df = pd.DataFrame()
        
        # Appending to combined DataFrame
        ndvi = pd.concat([ndvi, temp_chunk_df], ignore_index=True)

ndvi_grouped = ndvi.groupby(['site_id_created'])['ndvi'].mean().reset_index()
ndvi_grouped.to_csv(output_csv_path, index=False)

print("All chunks processed, combined results saved to:", output_csv_path)

### Combine them

In [ ]:
df_combined = (df
               .merge(tree_cover, on = 'site_id_created', how = 'left')
               .merge(other_land_cover, on = 'site_id_created', how = 'left')
               .merge(built_area_cover, on = 'site_id_created', how = 'left')
               .merge(road_length, on = 'site_id_created', how = 'left')
               .merge(forest_loss, on = 'site_id_created', how = 'left')
               .merge(elevation_slope, on = 'site_id_created', how = 'left')
               .merge(ndvi_top3, on = 'site_id_created', how = 'left')
               .merge(shadow_index_grouped, on = 'site_id_created', how = 'left')
               .merge(ndvi_grouped, on = 'site_id_created', how = 'left'))

In [ ]:
df_combined.info()